### 打开摄像头，检测移动物体，并用红线框出

In [3]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np  

#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture(0)  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
cv2.resizeWindow("DI-Video", 640, 480);


firstframe=None

while (camera.isOpened()):  

    ret,frame = camera.read()
    #print ret, frame
    
    if not ret:  
        break
        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if firstframe is None:  
        firstframe=gray  
        continue  
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
    #cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)  
    key = cv2.waitKey(1)
     
    if key == ord("q"):  
        break  

camera.release()  
cv2.destroyAllWindows()  


### 打开视频文件，检测移动物体

In [2]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np  

#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);


firstframe=None

count=-1
frame_list=[]

while (camera.isOpened()):  

    ret,frame = camera.read()
    #print ret, frame
    
    if not ret:  
        break

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)  
    key = cv2.waitKey(1)
     
    if key == ord("q"):  
        break  

camera.release()  
cv2.destroyAllWindows()  


uint8
uint8


In [16]:
a=[[1,2,3,4,5],[3,4,5,6,7]]
a.copy()

[[1, 2, 3, 4, 5], [3, 4, 5, 6, 7]]

### 从两幅图片提取特征点，并进行特征点的匹配

In [35]:
import cv2  
import numpy as np  
  
def drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch):  
    h1, w1 = img1_gray.shape[:2]  
    h2, w2 = img2_gray.shape[:2]  
  
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    vis[:h1, :w1] = img1_gray  
    vis[:h2, w1:w1 + w2] = img2_gray  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]  
  
    post1 = np.int32([kp1[pp].pt for pp in p1])  
    post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
  
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)  
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis)  
  
img1_gray = cv2.imread("moving3_pic1.JPG")  
img2_gray = cv2.imread("moving3_pic2.JPG")  
  
sift = cv2.xfeatures2d.SIFT_create()
#sift = cv2.SURF()  
  
kp1, des1 = sift.detectAndCompute(img1_gray, None)  
kp2, des2 = sift.detectAndCompute(img2_gray, None)  
  
# BFmatcher with default parms  
bf = cv2.BFMatcher(cv2.NORM_L2)  
matches = bf.knnMatch(des1, des2, k = 2)  
  
goodMatch = []  
for m,n in matches:  
    if m.distance < 0.5*n.distance:  
        goodMatch.append(m)  

#把这个函数改成官方函数以后，就跑不过去了
drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch[:20]) 

cv2.imshow("new",img3)  
cv2.waitKey(0)  
cv2.destroyAllWindows()

In [ ]:
import cv2  
import numpy as np  
  
def drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch):  
    h1, w1 = img1_gray.shape[:2]  
    h2, w2 = img2_gray.shape[:2]  
  
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    vis[:h1, :w1] = img1_gray  
    vis[:h2, w1:w1 + w2] = img2_gray  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]  
  
    post1 = np.int32([kp1[pp].pt for pp in p1])  
    post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
  
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)  
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis)  
  
img1_gray = cv2.imread("moving3_pic1.JPG")  
img2_gray = cv2.imread("moving3_pic2.JPG")  
  
sift = cv2.xfeatures2d.SIFT_create()
#sift = cv2.SURF()  
  
kp1, des1 = sift.detectAndCompute(img1_gray, None)  
kp2, des2 = sift.detectAndCompute(img2_gray, None)  
  
# BFmatcher with default parms  
bf = cv2.BFMatcher(cv2.NORM_L2)  
matches = bf.knnMatch(des1, des2, k = 2)  
  
goodMatch = []  
for m,n in matches:  
    if m.distance < 0.5*n.distance:  
        goodMatch.append(m)  

#把这个函数改成官方函数以后，就跑不过去了
drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch[:20]) 

cv2.imshow("new",img3)  
cv2.waitKey(0)  
cv2.destroyAllWindows()

### 从视频中获取特征点的移动轨迹，在右侧另开窗口显示


In [50]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np
import time
#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);



firstframe=None

count=-1
frame_list=[]
frame=None
frame_raw=None
last_frame=None
frame_match=None
frameDelta_1=None
frameDelta_2=None

def drawMatchesKnn_cv2_video(img1_gray,kp1,img2_gray,kp2,goodMatch,frameDelta):  
    #h1, w1 = img1_gray.shape[:2]  
    #h2, w2 = img2_gray.shape[:2] 
    #print(frameDelta)
    h1, w1 = frameDelta.shape[:2] 
    #print(h1,w1)
  
    #vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    #vis[:h1, :w1] = img1_gray  
    #vis[:h2, w1:w1 + w2] = img2_gray 
    vis = np.zeros((h1, w1 * 2 , 3), np.uint8)  
    vis[:h1, :w1] = img2_gray  
    vis[:h1, w1:w1 * 2 ] = frameDelta  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]
    
    #print(p1)
    #print(len(kp1))
  
    #post1 = np.int32([kp1[pp].pt for pp in p1])
    #post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
    post1 = np.int32([kp1[pp].pt for pp in p1])
    post2 = np.int32([kp2[pp].pt for pp in p2])
    
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(frameDelta, (x1, y1), (x2, y2), (0,0,255),thickness=1)
  
    for (x1, y1), (x2, y2) in zip(post1+(w1, 0), post2+(w1, 0)):  
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)  
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis) 
    return frameDelta



while (camera.isOpened()):  
    if frame_raw is not None:
        last_frame=frame_raw.copy()
    ret,frame = camera.read()
    if frame is not None:
        frame_raw=frame.copy()
    #print ret, frame
    
    if not ret:  
        break
    
    if count == -1:
        frameDelta_1=frame

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect

    if last_frame is not None and count%1==0 and len(hierarchy)!=0:
        #cv2.imwrite("shot"+str(count)+".png",last_frame)
        sift = cv2.xfeatures2d.SIFT_create() 
        kp1, des1 = sift.detectAndCompute(frame_raw, None)  
        kp2, des2 = sift.detectAndCompute(last_frame, None)
        #print(len(kp1))
        if len(kp1)!=0 and len(kp2)!=0:
            bf = cv2.BFMatcher(cv2.NORM_L2)  
            matches = bf.knnMatch(des1, des2, k = 2)  
            #print(len(matches))
            goodMatch = []  
            for m,n in matches:  
                if m.distance < 0.3*n.distance:  
                    goodMatch.append(m)
            #print(len(goodMatch))
            #frame_match = cv2.drawMatchesKnn(frame_raw,kp1,last_frame,kp2,goodMatch,None, flags=2) 
        #print(frameDelta_1)
        if goodMatch is not None:
            if frameDelta_2 is  None :
                frameDelta_2=drawMatchesKnn_cv2_video(frame_raw,kp1,last_frame,kp2,goodMatch[:5],frameDelta_1)
            else:
                frameDelta_2=drawMatchesKnn_cv2_video(frame_raw,kp1,last_frame,kp2,goodMatch[:5],frameDelta_2)
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)
    
    key = cv2.waitKey(1)
    if key == ord("q"):  
        break  
    
cv2.waitKey(0)          
camera.release()  
cv2.destroyAllWindows()  


改良

In [ ]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np
import time
#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);



firstframe=None

count=-1
frame_list=[]
frame=None
frame_raw=None
last_frame=None
frame_match=None
frameDelta_1=None
frameDelta_2=None

def drawMatchesKnn_cv2_video(img1_gray,kp1,img2_gray,kp2,goodMatch,frameDelta):  
    #h1, w1 = img1_gray.shape[:2]  
    #h2, w2 = img2_gray.shape[:2] 
    #print(frameDelta)
    h1, w1 = frameDelta.shape[:2] 
    #print(h1,w1)
  
    #vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    #vis[:h1, :w1] = img1_gray  
    #vis[:h2, w1:w1 + w2] = img2_gray 
    vis = np.zeros((h1, w1 * 2 , 3), np.uint8)  
    vis[:h1, :w1] = img2_gray  
    vis[:h1, w1:w1 * 2 ] = frameDelta  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]
    
    #print(p1)
    #print(len(kp1))
  
    #post1 = np.int32([kp1[pp].pt for pp in p1])
    #post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
    post1 = np.int32([kp1[pp].pt for pp in p1])
    post2 = np.int32([kp2[pp].pt for pp in p2])
    
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(frameDelta, (x1, y1), (x2, y2), (0,0,255),thickness=1)
    try :
        for (x1, y1), (x2, y2) in zip(post1+(w1, 0), post2+(w1, 0)):  
            cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)
    except ValueError as e :
        return frameDelta
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis) 
    return frameDelta



while (camera.isOpened()):  
    if frame_raw is not None:
        last_frame=frame_raw.copy()
    ret,frame = camera.read()
    if frame is not None:
        frame_raw=frame.copy()
    #print ret, frame
    
    if not ret:  
        break
    
    if count == -1:
        frameDelta_1=frame

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect

    if last_frame is not None and count%1==0 and len(hierarchy)!=0:
        #cv2.imwrite("shot"+str(count)+".png",last_frame)
        sift = cv2.xfeatures2d.SIFT_create() 
        kp1, des1 = sift.detectAndCompute(frame_raw, None)  
        kp2, des2 = sift.detectAndCompute(last_frame, None)
        #print(len(kp1))
        if len(kp1)!=0 and len(kp2)!=0:
            bf = cv2.BFMatcher(cv2.NORM_L2)  
            matches = bf.knnMatch(des1, des2, k = 2)  
            #print(len(matches))
            goodMatch = []  
            for m,n in matches:  
                if m.distance < 0.3*n.distance:  
                    goodMatch.append(m)
            #print(len(goodMatch))
            #frame_match = cv2.drawMatchesKnn(frame_raw,kp1,last_frame,kp2,goodMatch,None, flags=2) 
        #print(frameDelta_1)
        if goodMatch is not None:
            if frameDelta_2 is  None :
                frameDelta_2=drawMatchesKnn_cv2_video(frame_raw,kp1,last_frame,kp2,goodMatch[:10],frameDelta_1)
            else:
                frameDelta_2=drawMatchesKnn_cv2_video(frame_raw,kp1,last_frame,kp2,goodMatch[:10],frameDelta_2)
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)
    
    key = cv2.waitKey(1)
    if key == ord("q"):  
        break  
    
cv2.waitKey(0)          
camera.release()  
cv2.destroyAllWindows()  


uint8
uint8


从视频中寻找连续关键点

In [1]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np
import time
#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);



firstframe=None

count=-1
frame_list=[]
frame=None
frame_raw=None
last_frame=None
frame_match=None
frameDelta_1=None
frameDelta_2=None
continue_pt=None
goodMatch = []  

def drawMatchesKnn_cv2_video(img1_gray,kp1,img2_gray,kp2,goodMatch,frameDelta):  
    #h1, w1 = img1_gray.shape[:2]  
    #h2, w2 = img2_gray.shape[:2] 
    #print(frameDelta)
    h1, w1 = frameDelta.shape[:2] 
    #print(h1,w1)
  
    #vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    #vis[:h1, :w1] = img1_gray  
    #vis[:h2, w1:w1 + w2] = img2_gray 
    vis = np.zeros((h1, w1 * 2 , 3), np.uint8)  
    vis[:h1, :w1] = img2_gray  
    vis[:h1, w1:w1 * 2 ] = frameDelta  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]
    
    #print(p1)
    #print(len(kp1))
  
    #post1 = np.int32([kp1[pp].pt for pp in p1])
    #post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
    post1 = np.int32([kp1[pp].pt for pp in p1])
    post2 = np.int32([kp2[pp].pt for pp in p2])
    
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(frameDelta, (x1, y1), (x2, y2), (0,0,255),thickness=1)
  
    for (x1, y1), (x2, y2) in zip(post1+(w1, 0), post2+(w1, 0)):  
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)  
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis) 
    return frameDelta

def find_the_same(list1,list2):
    return set(list1) & set(list2)

while (camera.isOpened()):  
    if frame_raw is not None:
        last_frame=frame_raw.copy()
    ret,frame = camera.read()
    if frame is not None:
        frame_raw=frame.copy()
    #print ret, frame
    
    if not ret:  
        break
    
    if count == -1:
        frameDelta_1=frame

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect

    if last_frame is not None and count%3==0 and len(hierarchy)!=0:
        #cv2.imwrite("shot"+str(count)+".png",last_frame)
        sift = cv2.xfeatures2d.SIFT_create() 
        kp1, des1 = sift.detectAndCompute(frame_raw, None)  
        kp2, des2 = sift.detectAndCompute(last_frame, None)
        #print(len(kp1))
        if len(kp1)!=0 and len(kp2)!=0:
            bf = cv2.BFMatcher(cv2.NORM_L2)  
            matches = bf.knnMatch(des1, des2, k = 2)  
            #print(len(matches))
            
            for m,n in matches:  
                if m.distance < 1*n.distance:  
                    goodMatch.append(m)
         
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]
    print(p1)
    print(p2)

    if continue_pt is None :
        continue_pt = np.zeros((len(p1)), np.uint8) 
        for pp in p1:
            n=0
            np.append(continue_pt[n],kp1[pp])
        p1_last=p1
        continue
    print(p1_last)
    print(p2)
    the_same_index=find_the_same(p1_last,p2)
    p1_last=np.array(p1_last)
    p2=np.array(p2)
    if the_same_index is None:
        break
    print(the_same_index)
    for index in  the_same_index :
        print(index)
        index_raw=np.where( p2=index)
        index_last=np.where( p1_last=index)
        np.append(continue_pt[n],kp1[index_raw])
        
            
        
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)
    
    key = cv2.waitKey(1)
    if key == ord("q"):  
        break  
    
cv2.waitKey(0)          
camera.release()  
cv2.destroyAllWindows()  


uint8
uint8
[]
[]
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[]
[]
[]
[]
set()
[0, 1, 2]
[2, 0, 2]
[]
[2, 0, 2]
set()
[0, 1, 2]
[2, 0, 2]
[]
[2, 0, 2]
set()
[0, 1, 2]
[2, 0, 2]
[]
[2, 0, 2]
set()
[0, 1, 2, 0, 1, 2, 3, 4, 5]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
[]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
set()
[0, 1, 2, 0, 1, 2, 3, 4, 5]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
[]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
set()
[0, 1, 2, 0, 1, 2, 3, 4, 5]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
[]
[2, 0, 2, 2, 2, 2, 1, 0, 2]
set()
[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[2, 0, 2, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1, 1, 0, 3, 2, 3, 2, 3, 2, 2, 3, 4, 5, 3, 2, 5, 11, 12, 13, 14, 0, 1, 2, 3, 4, 7, 26, 9, 8, 9, 26, 11, 21, 22, 20, 19, 15, 11, 15, 11, 9, 8, 22, 11, 12, 13, 14, 45, 15, 26, 16, 17, 18, 19, 20, 21, 22, 23, 26, 24, 25, 26, 27, 28, 8, 25, 29, 30, 31, 33, 34, 27, 36, 36, 36, 38, 37, 41, 42, 42, 38, 39, 43, 44, 3, 5, 9, 41, 16, 27, 32, 18, 16, 16, 20, 44, 45, 51, 52, 53, 44

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
[2, 0, 2, 2, 2, 2, 1, 0, 2, 

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

...

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

[0, 1, 2, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

[]
[2, 0, 2, 2, 2, 2, 1, 0, 2, 0, 0, 1, 1, 1, 0, 3, 2, 3, 2, 3, 2, 2, 3, 4, 5, 3, 2, 5, 11, 12, 13, 14, 0, 1, 2, 3, 4, 7, 26, 9, 8, 9, 26, 11, 21, 22, 20, 19, 15, 11, 15, 11, 9, 8, 22, 11, 12, 13, 14, 45, 15, 26, 16, 17, 18, 19, 20, 21, 22, 23, 26, 24, 25, 26, 27, 28, 8, 25, 29, 30, 31, 33, 34, 27, 36, 36, 36, 38, 37, 41, 42, 42, 38, 39, 43, 44, 3, 5, 9, 41, 16, 27, 32, 18, 16, 16, 20, 44, 45, 51, 52, 53, 44, 55, 56, 56, 59, 60, 61, 60, 61, 62, 57, 63, 63, 64, 55, 65, 66, 1, 2, 2, 5, 6, 8, 6, 7, 10, 11, 12, 13, 14, 15, 15, 17, 18, 20, 21, 23, 22, 24, 27, 29, 26, 30, 31, 32, 34, 36, 35, 13, 5, 23, 0, 23, 1, 2, 5, 7, 13, 7, 9, 10, 11, 12, 5, 13, 33, 15, 16, 17, 18, 19, 20, 22, 23, 37, 37, 25, 26, 28, 33, 20, 30, 31, 32, 33, 33, 36, 12, 12, 23, 39, 59, 40, 40, 41, 42, 43, 43, 45, 45, 46, 47, 51, 50, 50, 51, 51, 54, 52, 54, 55, 56, 57, 59, 0, 30, 5, 6, 14, 15, 16, 25, 8, 17, 18, 7, 8, 22, 34, 24, 23, 22, 41, 27, 31, 28, 29, 30, 33, 32, 32, 34, 36, 37, 37, 38, 40, 41, 1, 2, 2, 3, 15, 0, 1, 

关于这个函数的官方文档：
http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html#matcher

关于这个函数的bug的讨论：
https://stackoverflow.com/questions/31631352/typeerror-required-argument-outimg-pos-6-not-found

https://github.com/opencv/opencv/issues/4237

In [3]:
import numpy as np
x=np.zeros((3,2),np.int8)
print(x)
x[0]=[1,2,3]
# x[0]=np.append(x[0],1)
print(x)




[[0 0]
 [0 0]
 [0 0]]


ValueError: cannot copy sequence with size 3 to array axis with dimension 2